In [28]:
import requests
import datetime as dt
import numpy as np
import pandas as pd
import requests
from sklearn_extra.cluster import KMedoids # https://scikit-learn-extra.readthedocs.io/en/stable/_modules/sklearn_extra/cluster/_k_medoids.html#KMedoids

start = "2022-01-01T00:00"
# end = (dt.datetime.now()+dt.timedelta(days=5)).strftime("%Y-%m-%dT00:00")
end = "2024-01-01T00:00"
start2 = (dt.datetime.strptime(start, "%Y-%m-%dT%H:%M")+ dt.timedelta(hours=0)).strftime("%Y-%m-%dT%H:%M")

In [29]:
responseFCR = requests.get(url=f"https://api.energidataservice.dk/dataset/FcrNdDK2?offset=0&start={start}&end={end}&filter=%7B%22PriceArea%22:[%22DK2%22],%22AuctionType%22:[%22D-1%20early%22,%22D-1%20late%22],%22ProductName%22:[%22FCR-D%20ned%22,%22FCR-d%20upp%22]%7D&sort=HourDK%20ASC")
responseSpot = requests.get(url=f"https://api.energidataservice.dk/dataset/Elspotprices?offset=0&start={start}&end={end}&filter=%7B%22PriceArea%22:[%22DK2%22]%7D&sort=HourUTC%20ASC")

In [30]:
result = {}
result["timestamp"] = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
result["status_code"] = responseFCR.status_code
if responseFCR.status_code == 200:
    result["dataFCR"] = responseFCR.json()["records"]
    dfDataFCR = pd.DataFrame.from_records(result["dataFCR"])
if responseSpot.status_code == 200:
    result["dataSpot"] = responseSpot.json()["records"]
    dfDataSpot = pd.DataFrame.from_records(result["dataSpot"])

In [31]:
# Convert 'HourDK' to DateTime format
dfDataFCR['HourDK'] = pd.to_datetime(dfDataFCR['HourDK'], format='ISO8601')

# Group by 'HourDK', 'PriceArea', 'ProductName', and 'AuctionType', then pivot the table
grouped = dfDataFCR.groupby(['HourDK', 'PriceArea', 'ProductName', 'AuctionType'])['PriceTotalEUR'].sum().reset_index()
pivot_table = grouped.pivot_table(index=['HourDK', 'PriceArea'], columns=['ProductName', 'AuctionType'], values='PriceTotalEUR').reset_index()

# Flatten the multi-index columns
pivot_table.columns = pivot_table.columns.map(lambda x: f'{x[0]}, {x[1]}' if x[1] else x[0])

# Create a range of dates with hourly frequency
date_range = pd.date_range(start=start2, periods=len(pivot_table), freq='h')

pivot_table['Timestamp'] = pd.DataFrame({'Timestamp': date_range})
pivot_table['Date'] = pivot_table['Timestamp'].dt.date
pivot_table['Hour'] = pivot_table['Timestamp'].dt.hour
dfEU = pivot_table.pivot_table(index=['Date'], columns=['Hour'], values='FCR-D upp, D-1 early', fill_value=0, aggfunc='sum', dropna=False).reset_index()
dfLU = pivot_table.pivot_table(index=['Date'], columns=['Hour'], values='FCR-D upp, D-1 late', fill_value=0, aggfunc='sum', dropna=False).reset_index()
dfED = pivot_table.pivot_table(index=['Date'], columns=['Hour'], values='FCR-D ned, D-1 early', fill_value=0, aggfunc='sum', dropna=False).reset_index()
dfLD = pivot_table.pivot_table(index=['Date'], columns=['Hour'], values='FCR-D ned, D-1 late', fill_value=0, aggfunc='sum', dropna=False).reset_index()

In [32]:
# Create a range of dates with hourly frequency
date_range = pd.date_range(start=start2, periods=len(dfDataSpot), freq='h')
dfDataSpot['Timestamp'] = pd.DataFrame({'Timestamp': date_range})
dfDataSpot['Date'] = dfDataSpot['Timestamp'].dt.date
dfDataSpot['Hour'] = dfDataSpot['Timestamp'].dt.hour
dfSpot = dfDataSpot.pivot_table(index="Date", columns=['Hour'], values='SpotPriceEUR', fill_value=0, aggfunc='sum', dropna=False).reset_index()

In [33]:
# Combine DataFrames to 120-axis vectors for 
dfEarly = dfEU.merge(dfED, on='Date', suffixes=['_EU', '_ED'])
dfLate = dfLU.merge(dfLD, on='Date', suffixes=['_LU', '_LD'])
dfAll = dfEarly.merge(dfSpot, on='Date').merge(dfLate, on='Date')

In [34]:
dfAll.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\pricevectors.csv", index=False)

In [35]:
npAll = dfAll.to_numpy()
npAll = np.delete(npAll, obj=0, axis=1)

In [44]:
dim = len(dfAll)
npDist = np.zeros((dim,dim))
for i in range(dim):
    for j in range(dim):
        v1 = dfAll.loc[i, '0_EU':'23_LD'].to_list()
        v2 = dfAll.loc[j, '0_EU':'23_LD'].to_list()
        if i == j:
            npDist[i,j] = 0
        elif i < j:
            npDist[i,j] = np.sqrt(sum((float(v1[k])-float(v2[k]))**2 for k in range(len(v1))))
        else:
            npDist[i,j] = npDist[j,i]

In [37]:
dates = dfAll['Date'].to_list()
dfDist = pd.DataFrame(npDist, columns=dates)
dfDist.insert(0,'Date',dates)
dfDist.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\distances.csv", index=False)

In [38]:
kmedoids_100 = KMedoids(n_clusters=100, random_state=0, metric='precomputed', max_iter=1000, method='pam').fit(npDist)
npProb = np.array([np.sum(kmedoids_100.labels_ == i)/len(kmedoids_100.labels_) for i in range(100)])
npDist_100 = npDist[kmedoids_100.medoid_indices_][:,kmedoids_100.medoid_indices_]

In [39]:
npLate = npAll[kmedoids_100.medoid_indices_.tolist()][:,72:]
npLU = npAll[kmedoids_100.medoid_indices_.tolist()][:,72:96]
npLD = npAll[kmedoids_100.medoid_indices_.tolist()][:,96:120]

In [40]:
kmedoids_10 = KMedoids(n_clusters=10, random_state=0, metric='precomputed', max_iter=1000, method='pam').fit(npDist_100)
npDist_10 = npDist[kmedoids_10.medoid_indices_][:,kmedoids_10.medoid_indices_]
lst = []
for i in kmedoids_10.labels_.tolist():
    j = (kmedoids_10.medoid_indices_.tolist())[i]
    lst.append((kmedoids_100.medoid_indices_.tolist())[j])
npSpot = npAll[lst][:,48:72]
npGroup = kmedoids_10.labels_.tolist()

In [41]:
kmedoids_1 = KMedoids(n_clusters=1, random_state=0, metric='precomputed', max_iter=0, method='pam').fit(npDist_10)
idx = kmedoids_100.medoid_indices_[kmedoids_10.medoid_indices_[kmedoids_1.medoid_indices_[0]]]
npEarly = npAll[[idx for i in range(100)]][:,0:48]
npEU = npAll[[idx for i in range(100)]][:,0:24]
npED = npAll[[idx for i in range(100)]][:,24:48]

In [42]:
# np arrays to DFs
dfPricesEarly = pd.DataFrame(npEarly, index=None)
dfPricesSpot = pd.DataFrame(npSpot, index=None)
dfPricesLate = pd.DataFrame(npLate, index=None)
dfProbabilities = pd.DataFrame(npProb, index=None)
dfGroup = pd.DataFrame(npGroup, index=None)


dfPricesEarly.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\PricesEarly.csv", index=False)
dfPricesSpot.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\PricesSpot.csv", index=False)
dfPricesLate.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\PricesLate.csv", index=False)
dfProbabilities.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\Probabilities.csv", index=False)
dfGroup.to_csv(r"C:\Users\kmo\OneDrive - Better Energy\Documents\Python\Speciale\Data\Group.csv", index=False)